In [18]:
import os
from dotenv import load_dotenv

load_dotenv()

True

Load the environment variables in a class


In [19]:
class Config:
    def __init__(self):
        self.open_api_key = os.getenv('OPENAI_API_KEY')
        self.openweather_api_key = os.getenv('OPENWEATHER_API_KEY')
        self.exchange_rate_api_key = os.getenv('EXCHANGE_RATE_API_KEY')
        self.google_places_api_key = os.getenv('GOOGLE_PLACES_API_KEY')
        self.serpapi_key = os.getenv('SERPAPI_KEY')
        self.serper_api_key = os.getenv('SERPER_API_KEY')

Define the TravelPlanner class initialization and basic setup

In [69]:
from typing import Dict, List, Any,Literal
import requests
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import  SerpAPIWrapper, GoogleSerperAPIWrapper
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langgraph.graph import MessagesState, StateGraph, END, START
from langgraph.prebuilt import ToolNode, tools_condition

In [14]:
class WeatherService:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "http://api.openweathermap.org/data/2.5"
        print(f"WeatherService initialized with API key: {self.api_key}")
    
    def get_current_weather(self, city: str) -> Dict:
        """Get current weather for a city"""
        try:
            url = f"{self.base_url}/weather"
            params = {
                "q": city,
                "appid": self.api_key,
                "units": "metric"
            }
            response = requests.get(url, params=params)
            print(response)
            return response.json() if response.status_code == 200 else {}
        except:
            return {}
    
    def get_weather_forecast(self, city: str, days: int = 5) -> Dict:
        """Get weather forecast for a city"""
        try:
            url = f"{self.base_url}/forecast"
            params = {
                "q": city,
                "appid": self.api_key,
                "units": "metric",
                "cnt": days * 8  # 8 forecasts per day (3-hour intervals)
            }
            response = requests.get(url, params=params)
            return response.json() if response.status_code == 200 else {}
        except:
            return {}


In [ ]:
# Unit testing for weather service
conf = Config()
weather_service = WeatherService(conf.openweather_api_key)
weather_service.get_current_weather("jabalpur")

WeatherService initialized with API key: 60039f540fdf7d831a97ac117a02fd9a
<Response [200]>


{'coord': {'lon': 79.9501, 'lat': 23.167},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 35.04,
  'feels_like': 36.71,
  'temp_min': 35.04,
  'temp_max': 35.04,
  'pressure': 1000,
  'humidity': 38,
  'sea_level': 1000,
  'grnd_level': 954},
 'visibility': 10000,
 'wind': {'speed': 5.26, 'deg': 251, 'gust': 5.69},
 'clouds': {'all': 54},
 'dt': 1750232141,
 'sys': {'country': 'IN', 'sunrise': 1750204476, 'sunset': 1750253281},
 'timezone': 19800,
 'id': 1269633,
 'name': 'Jabalpur',
 'cod': 200}

In [50]:
class CurrencyService:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://api.exchangerate-api.com/v4/latest"
        print(f"CurrencyService initialized with API key: {self.api_key}")

    def get_exchange_rate(self, from_currency: str, to_currency: str) -> float:
        """Get exchange rate from one currency to another"""
        try:
            url = f"{self.base_url}/{from_currency}"
            response = requests.get(url)
            data = response.json()
            print(response.status_code)
            print(to_currency in data['rates'])
            if response.status_code == 200 and to_currency in data['rates']:
                print("inside response")
                
                return data["rates"].get(to_currency, 0.0)
            return "1.0"
            
        except:
            return "default value"
    
    def convert_currency(self, amount: float, from_currency: str, to_currency: str) -> float:
        """Convert amount from one currency to another"""
        try:
            rate = self.get_exchange_rate(from_currency, to_currency)
            return amount * rate
        except:
            return 0.0


In [52]:
#Unit testing for currency service
currency_service = CurrencyService(conf.exchange_rate_api_key)
currency_service.convert_currency(5.09,"USD", "INR")

CurrencyService initialized with API key: 71d149cabe4fe6ad19591188
200
True
inside response


439.4197

In [53]:
class TravelCalculator:
    @staticmethod
    def add(a: float, b: float) -> float:
        """Add two numbers"""
        return a + b
    
    @staticmethod
    def multiply(a: float, b: float) -> float:
        """Multiply two numbers"""
        return a * b
    
    @staticmethod
    def calculate_total_cost(*costs: float) -> float:
        """Calculate total cost from multiple expenses"""
        return sum(costs)
    
    @staticmethod
    def calculate_daily_budget(total_cost: float, days: int) -> float:
        """Calculate daily budget"""
        return total_cost / days if days > 0 else 0

In [64]:
class TravelPlanner:
    def __init__(self, config: Config):
        self.config = config
        self.weather_service = WeatherService(config.openweather_api_key)
        self.currency_service = CurrencyService(config.exchange_rate_api_key)
        self.calculator = TravelCalculator()
    
        #Initialize search tools with real-time capabilities
        self.search_tool = DuckDuckGoSearchRun()

        # Try SerpAPI for fresh Google results
        # Initialize SerpAPI for real-time Google search results
        try:
            if config.serpapi_key:
                self.serp_search = SerpAPIWrapper(serpapi_api_key=config.serpapi_key)
            else:
                self.serp_search = None
        except Exception:
            self.serp_search = None

        # Initialize Google Serper for real-time Google search results
        try:
            if config.serper_api_key:
                self.google_serper_search = GoogleSerperAPIWrapper(serper_api_key=config.serper_api_key)
            else:
                self.google_serper_search = None
        except Exception:
            self.google_serper_search = None

        # Initialize LLM
        self.llm = ChatOpenAI(
            model="gpt-4.1-2025-04-14",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            # openai_api_base=os.getenv("BASE_URL"),
        )
        # Setup tools
        self.tools = self._setup_tools()
        self.llm_with_tools = self.llm.bind_tools(self.tools)
    
    def _setup_tools(self) -> List[Any]:
        """Setup tools for the travel planner"""
        @tool
        def search_attractions(city: str) -> str:
            """Search for top attractions in a city using real time data"""
            query = f"top attractions activities things to do in {city}"

            # Try SerpAPI for fresh Google results
            if self.serp_search:
                try:
                    serp_result = self.serp_search.run(query)
                    if serp_result and len(serp_result) > 50:
                        return f"Latest search results: {serp_result}"
                except Exception:
                    pass
            
            # Try Google Serper
            if self.serper_search:
                try:
                    serper_result = self.serper_search.run(query)
                    if serper_result and len(serp_result) > 50:
                        return f"Current search data: {serper_result}"
                except Exception:
                    pass
            # Fallback to DuckDuckGo
            return self.search_tool.invoke(query)
        
        @tool
        def search_restaurants(city: str) -> str:
            """Search for restaurants in a city using DuckDuckGo"""
            query = f" top rated restaurants dining places to eat in {city}"
            #Try SerpAPI for fresh Google results
            if self.serp_search:
                try:
                    serp_result = self.serp_search.run(query)
                    if serp_result and len(serp_result) > 50:
                        return f"Latest search results: {serp_result}"
                except Exception:
                    pass
            # Fallback to DuckDuckGo
            return self.search_tool.invoke(query)
        
        @tool
        def search_transportation(city: str) -> str:
            """Search for transportation options in a city using real-time data"""
            query = f"transportation options getting around {city} public transport taxi uber  rental car trains"
            if self.serp_search:
                try:
                    serp_result = self.serp_search.run(query)
                    if serp_result and len(serp_result) > 50:
                        return f"Latest search results: {serp_result}"
                except Exception:
                    pass
            # Fallback to DuckDuckGo
            return self.search_tool.invoke(query)

        @tool
        def search_activities(city: str) -> str:
            """Search for things to do in a city using real-time data"""
            query = f"activities things to do in {city}"
            if self.google_serper_search:
                return self.google_serper_search.run(query)
            return "No real-time data available"
        
        @tool
        def get_current_weather(city: str) -> str:
            """Get current weather for a city"""
            weather_data = self.weather_service.get_current_weather(city)
            if weather_data:
                temp = weather_data.get('main', {}).get('temp', 'N/A')
                desc = weather_data.get('weather', [{}])[0].get('description', 'N/A')
                return f"Current weather in {city}: {temp}°C, {desc}"
            return f"Could not fetch weather for {city}"
        
        @tool
        def get_weather_forecast(city: str, days: int = 5) -> str:
            """Get weather forecast for a city"""
            forecast_data = self.weather_service.get_weather_forecast(city, days)
            if forecast_data and 'list' in forecast_data:
                forecast_summary = []
                for i in range(0, min(len(forecast_data['list']), days * 8), 8):
                    item = forecast_data['list'][i]
                    date = item['dt_txt'].split(' ')[0]
                    temp = item['main']['temp']
                    desc = item['weather'][0]['description']
                    forecast_summary.append(f"{date}: {temp}°C, {desc}")
                return f"Weather forecast for {city}:\n" + "\n".join(forecast_summary)
            return f"Could not fetch forecast for {city}"
        
        @tool
        def search_hotels(city: str, budget_range: str = "mid-range") -> str:
            """Search for hotels in a city with budget range using real-time data"""
            query = f"{budget_range} hotels accommodation {city} price per night booking availability"
            # Try SerpAPI for real-time hotel prices and availability
            if self.serp_search:
                try:
                    serp_result = self.serp_search.run(query)
                    if serp_result and len(serp_result) > 50:
                        return f"Real-time hotel data: {serp_result}"
                except Exception:
                    pass
            
            # Try Google Serper
            if self.serper_search:
                try:
                    serper_result = self.serper_search.run(query)
                    if serper_result and len(serp_result) > 50:
                        return f"Latest hotel availability: {serper_result}"
                except Exception:
                    pass
            
            # Fallback to DuckDuckGo
            return self.search_tool.invoke(query)
        
        @tool
        def estimate_hotel_cost(price_per_night: float, total_days: int) -> float:
            """Calculate total hotel cost"""
            return self.calculator.multiply(price_per_night, total_days)
        
        @tool
        def add_costs(cost1: float, cost2: float) -> float:
            """Add two costs together"""
            return self.calculator.add(cost1, cost2)
        
        @tool
        def multiply_costs(cost: float, multiplier: float) -> float:
            """Multiply cost by a multiplier"""
            return self.calculator.multiply(cost, multiplier)
        
        @tool
        def calculate_total_expense(*costs: float) -> float:
            """Calculate total expense from multiple costs"""
            return self.calculator.calculate_total_cost(*costs)
        
        @tool
        def calculate_daily_budget(total_cost: float, days: int) -> float:
            """Calculate daily budget"""
            return self.calculator.calculate_daily_budget(total_cost, days)
        
        @tool
        def get_exchange_rate(from_currency: str, to_currency: str) -> float:
            """Get exchange rate between currencies"""
            return self.currency_service.get_exchange_rate(from_currency, to_currency)
        
        @tool
        def convert_currency(amount: float, from_currency: str, to_currency: str) -> float:
            """Convert amount from one currency to another"""
            return self.currency_service.convert_currency(amount, from_currency, to_currency)
        
        @tool
        def create_day_plan(city: str, day_number: int, attractions: str, weather: str) -> str:
            """Create a day plan for the trip"""
            return f"Day {day_number} in {city}:\n" \
                   f"Weather: {weather}\n" \
                   f"Recommended activities: {attractions[:200]}...\n" \
                   f"Tips: Plan indoor activities if weather is poor."
        
        return [
            search_attractions, search_restaurants, search_transportation,
            get_current_weather, get_weather_forecast, search_hotels,
            estimate_hotel_cost, add_costs, multiply_costs, calculate_total_expense,
            calculate_daily_budget, get_exchange_rate, convert_currency, create_day_plan,search_activities
        ]
        

In [80]:
class TravelAgent:
    def __init__(self, travel_planner: TravelPlanner):
        self.travel_planner = travel_planner
        self.system_prompt = """

                You are an intelligent AI Travel Agent and Expense Planner, designed to help users plan trips to any city around the world using real-time data.

                **IMPORTANT:** Always deliver a **comprehensive and detailed** travel plan in a single response. Do **not** say things like *“I’ll prepare”* or *“hold on”* — instead, provide all the information immediately.

                Your response must include:

                * A full day-by-day itinerary
                * Specific attractions with descriptions
                * Restaurant suggestions with price estimates
                * A detailed cost breakdown
                * Transportation options and info
                * Weather forecasts and summaries

                Leverage the available tools to retrieve real-time data and perform accurate calculations. Format the entire response in **clear, structured Markdown** for easy reading.
                for example:
                Here's an example of a complete response from your AI Travel Agent, formatted in clean **Markdown**:

                # ✈️ Trip Plan: 3-Day Getaway to **Paris, France**

                ## 📅 Itinerary Overview

                **Dates:** June 21 – June 23, 2025
                **Travelers:** 2 Adults
                **Currency:** EUR

                ---

                ## 🌤️ Weather Forecast (Paris)

                | Date    | Forecast      | Temperature |
                | ------- | ------------- | ----------- |
                | June 21 | Sunny         | 24°C        |
                | June 22 | Partly Cloudy | 22°C        |
                | June 23 | Light Rain    | 19°C        |

                ---

                ## 🗺️ Day-by-Day Itinerary

                ### **Day 1: Explore Historic Paris**

                * 🗼 **Eiffel Tower** (Entry: €18/person)

                * Visit the summit and enjoy panoramic views.
                * 🚶‍♂️ **Seine River Walk**

                * Scenic walk from Eiffel to Notre-Dame.
                * 🥐 **Lunch:** *Le Petit Cler* (€20/person)
                * 🖼️ **Louvre Museum** (Entry: €17/person)

                * Explore Mona Lisa, Egyptian artifacts, and more.
                * 🍷 **Dinner:** *Chez Janou* (€35/person)

                ---

                ### **Day 2: Montmartre & Local Vibes**

                * 🚇 **Morning:** Metro to Montmartre
                * 🎨 **Visit:** *Sacré-Cœur* & local artist square
                * ☕ **Brunch:** *Hardware Société* (€25/person)
                * 🛍️ **Afternoon:** Stroll Rue des Martyrs boutiques
                * 🎶 **Evening Show:** *Moulin Rouge* (€80/person)
                * 🍽️ **Dinner:** *Le Refuge des Fondus* (€30/person)

                ---

                ### **Day 3: Palace Day Trip**

                * 🚌 **Day Tour:** *Versailles Palace & Gardens*

                * Tour & transport: €65/person
                * 🥗 **Lunch:** Garden Café at Versailles (€20/person)
                * 🛒 **Evening:** Return for souvenir shopping at Galeries Lafayette
                * 🍕 **Dinner:** *Pink Mamma* (€35/person)

                ---

                ## 🏨 Hotel Recommendation

                **Hotel:** Hotel Le Six (4-star, central location)
                **Rate:** €150/night × 3 nights = **€450 total**

                ---

                ## 💰 Estimated Trip Cost Breakdown (EUR)

                | Category       | Cost (for 2 people) |
                | -------------- | ------------------- |
                | Flights (est.) | €400                |
                | Hotel          | €450                |
                | Attractions    | €295                |
                | Food & Dining  | €330                |
                | Transportation | €80                 |
                | Misc. Shopping | €100                |
                | **Total**      | **€1,655**          |

                ---

                ## 🚆 Transportation Info

                * **Airport to city:** RER B train (€10/person)
                * **Daily travel:** Paris Métro pass (€14.90/day for 2 zones)
                * **To Versailles:** RER C train or tour shuttle included in day trip

                ---

                ## ✅ Summary

                You’re all set for a memorable 3-day escape to **Paris**! With stunning landmarks, vibrant culture, and delectable cuisine, this itinerary balances sightseeing, relaxation, and local experiences.

                *Bon voyage! 🌍✈️*"""
        # Build the graph with proper termination
        self.graph = self._build_graph()

    def _build_graph(self) -> StateGraph:
        """Build the graph with proper termination"""
        
        def agentstate(state:MessagesState):
            """Agent state to handle messages"""
            user_question = state["messages"]
            input_question = [self.system_prompt] + user_question
            response = self.travel_planner.llm_with_tools.invoke(input_question)
            return {"messages": [response]}

        def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
            """Enhanced decision function for when to continue or end"""
            last_message = state["messages"][-1]
            
            # If it's a tool call, continue to tools
            if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
                return "tools"
            
            # Check if response seems complete
            content = last_message.content.lower()
            
            # Indicators that we need more information
            incomplete_phrases = [
                "let me search",
                "i'll look up",
                "please hold on",
                "i'll prepare",
                "let me gather",
                "i need to check"
            ]
            
            # If response contains incomplete phrases, continue
            if any(phrase in content for phrase in incomplete_phrases):
                return "tools"
            
            # Check if response is too short (likely incomplete)
            if len(last_message.content) < 500:
                return "tools"
            
            # Check if we have essential travel info
            essential_keywords = ["hotel", "attraction", "cost", "weather", "itinerary"]
            has_essential_info = sum(1 for keyword in essential_keywords if keyword in content) >= 3
            
            if not has_essential_info:
                return "tools"
            
            # Response seems complete, end the workflow
            return "__end__"
        
        # Create workflow with enhanced control
        workflow = StateGraph(MessagesState)
        workflow.add_node("agent", agentstate)
        workflow.add_node("tools", ToolNode(self.travel_planner.tools))
        # Add edges with better control
        workflow.add_edge(START, "agent")
        workflow.add_conditional_edges("agent", should_continue)
        workflow.add_edge("tools", "agent")

        # Compile with recursion limit to prevent infinite loops
        return workflow.compile()
    
    def plan_trip(self, user_input: str, max_iterations: int = 10) -> str:
        """Main function to plan a trip with iteration control"""
        messages = [HumanMessage(content=user_input)]
        
        # Add iteration counter to prevent infinite loops
        config = {"recursion_limit": max_iterations}
        
        try:
            response = self.graph.invoke({"messages": messages}, config=config)
            final_response = response["messages"][-1].content
            
            # Final check - if still incomplete, force a summary
            if len(final_response) < 800:
                summary_prompt = f"""
                Based on all the information gathered, provide a COMPLETE travel summary now. 
                Don't use tools anymore. Use the information you have to create a comprehensive plan.
                Format your response in clean Markdown with proper headers, lists, and formatting.
                Original request: {user_input}
                """
                
                summary_messages = response["messages"] + [HumanMessage(content=summary_prompt)]
                final_response_obj = self.travel_planner.llm_with_tools.invoke(summary_messages)
                return final_response_obj.content
            
            return final_response
            
        except Exception as e:
            print(f"Workflow error: {e}")
            # Fallback - direct LLM call
            return self._fallback_planning(user_input)

    def _fallback_planning(self, user_input: str) -> str:
        """Fallback method if workflow fails"""
        fallback_prompt = f"""
        Create a complete travel plan for: {user_input}
        
        Provide a comprehensive response including:
        - Daily itinerary
        - Top attractions
        - Restaurant recommendations  
        - Cost estimates
        - Weather information
        - Transportation details
        
        Format your response in clean Markdown with proper headers, lists, and formatting.
        Use your knowledge to provide helpful estimates even without real-time data.
        """
        
        messages = [self.system_prompt, HumanMessage(content=fallback_prompt)]
        response = self.travel_planner.llm_with_tools.invoke(messages)
        return response.content

    def export_to_markdown(self, response_text: str, filename: str = "travel_plan.md") -> str:
        """Export travel plan to Markdown file with proper formatting"""
        from datetime import datetime
        
        # Create markdown content with metadata header
        markdown_content = f"""# 🌍 AI Travel Plan

        **Generated:** {datetime.now().strftime('%Y-%m-%d at %H:%M')}  
        **Created by:** Himani's Re-Act agent

        ---

        {response_text}

        ---

        *This travel plan was generated by AI. Please verify all information, especially prices, operating hours, and travel requirements before your trip.*
        """
        
        try:
            # Write to markdown file with UTF-8 encoding
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(markdown_content)
            
            print(f"✅ Markdown file saved as: {filename}")
            return filename
            
        except Exception as e:
            print(f"❌ Error saving markdown file: {e}")
            return None
      

In [78]:
config = Config()
travel_planner = TravelPlanner(config)
travel_agent = TravelAgent(travel_planner)

WeatherService initialized with API key: 60039f540fdf7d831a97ac117a02fd9a
CurrencyService initialized with API key: 71d149cabe4fe6ad19591188


In [81]:
def main():
    """Main function with clean workflow and Markdown export"""
   
    config = Config()
    travel_planner = TravelPlanner(config)
    travel_agent = TravelAgent(travel_planner)
   
    # More specific queries for better results
    example_queries = [
        "give me  a detailed 1-day trip itenary to udaipur including best view hotels historic places best food restaurants with  budget of 10000 INR  i need budget converted to indian rupees"
    ]
   
    print("🌍 AI Travel Agent - Enhanced Workflow with Markdown Export 🌍")
    print("=" * 60)
   
    for i, query in enumerate(example_queries, 1):
        print(f"\n📝 Processing Query {i}:")
        print(f"Request: {query}")
        print(f"\n🤖 Generating travel plan...")
       
        try:
            # Use enhanced planning method
            response = travel_agent.plan_trip(query, max_iterations=10)
           
            # Export to Markdown directly - no console spam
            from datetime import datetime
            filename = f"AI_travel_plan_{i}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
            saved_file = travel_agent.export_to_markdown(response, filename)
           
            if saved_file:
                print(f"✅ Travel plan generated successfully!")
                print(f"📄 Saved as: {saved_file}")
                print(f"📊 Plan length: {len(response)} characters")
            else:
                print("❌ Failed to save markdown file")
           
        except Exception as e:
            print(f"❌ Error generating travel plan: {str(e)}")
       
        print("\n" + "=" * 60)

if __name__ == "__main__":
    main()

WeatherService initialized with API key: 60039f540fdf7d831a97ac117a02fd9a
CurrencyService initialized with API key: 71d149cabe4fe6ad19591188
🌍 AI Travel Agent - Enhanced Workflow with Markdown Export 🌍

📝 Processing Query 1:
Request: give me  a detailed 1-day trip itenary to udaipur including best view hotels historic places best food restaurants with  budget of 10000 INR  i need budget converted to indian rupees

🤖 Generating travel plan...
<Response [200]>
✅ Markdown file saved as: AI_travel_plan_1_20250618_144010.md
✅ Travel plan generated successfully!
📄 Saved as: AI_travel_plan_1_20250618_144010.md
📊 Plan length: 3856 characters

